In [ ]:
#!/bin/bash
# train_broadcast_memory.sh - Run on RunPod

# Clone repo
!git clone https://github.com/Eran-BA/PoT.git
%cd PoT



In [ ]:
# Install dependencies
!pip install torch torchvision torchaudio
!pip install tqdm numpy huggingface_hub wandb



In [ ]:
!pip install -U typing_extensions
# Or specifically:
# pip install typing_extensions>=4.7.1


In [ ]:
# Login to Weights & Biases
import wandb
wandb.login()


In [ ]:

!python experiments/sudoku_poh_benchmark.py \
    --d-model 512 \
    --d-ff 2048 \
    --model hybrid \
    --controller transformer \
    --d-ctrl 256 \
    --max-depth 32 \
    --injection-mode broadcast_memory \
    --injection-memory-size 16 \
    --injection-n-heads 4 \
    --epochs 1500 \
    --batch-size 768 \
    --lr 3e-4 \
    --warmup-steps 2000 \
    --d-model 512 \
    --n-heads 8 \
    --H-cycles 2 \
    --L-cycles 6 \
    --H-layers 2 \
    --L-layers 2 \
    --hrm-grad-style \
    --halt-max-steps 2 \
    --eval-interval 25 \
    --dropout 0.039 \
    --wandb \
    --project feature-injection-ablation \
    --run-name broadcast-memory-1500ep \
    --download


In [ ]:
!python experiments/sudoku_poh_benchmark.py \
    --d-model 512 \
    --d-ff 2048 \
    --model hybrid \
    --controller transformer \
    --d-ctrl 256 \
    --max-depth 32 \
    --injection-mode broadcast_memory \
    --injection-memory-size 16 \
    --injection-n-heads 4 \
    --epochs 3000 \
    --batch-size 768 \
    --lr 3e-4 \
    --warmup-steps 2000 \
    --n-heads 8 \
    --H-cycles 2 \
    --L-cycles 6 \
    --H-layers 2 \
    --L-layers 2 \
    --hrm-grad-style \
    --halt-max-steps 3 \
    --eval-interval 25 \
    --dropout 0.039 \
    --wandb \
    --project feature-injection-ablation \
    --run-name broadcast-memory-halt3 \
    --resume "eranbt92-open-university-of-israel/feature-injection-ablation/hybrid-transformer-best:latest" \
    --download

In [ ]:
!python experiments/sudoku_poh_benchmark.py \
    --d-model 512 \
    --d-ff 2048 \
    --model hybrid \
    --controller transformer \
    --d-ctrl 256 \
    --max-depth 32 \
    --injection-mode broadcast_memory \
    --injection-memory-size 16 \
    --injection-n-heads 4 \
    --epochs 3000 \
    --batch-size 768 \
    --lr 3e-4 \
    --warmup-steps 2000 \
    --n-heads 8 \
    --H-cycles 2 \
    --L-cycles 6 \
    --H-layers 2 \
    --L-layers 2 \
    --hrm-grad-style \
    --halt-max-steps 4 \
    --eval-interval 25 \
    --dropout 0.039 \
    --wandb \
    --project feature-injection-ablation \
    --run-name broadcast-memory-halt4 \
    --resume "eranbt92-open-university-of-israel/feature-injection-ablation/hybrid-transformer-best:latest" \
    --download

In [ ]:
!python experiments/sudoku_poh_benchmark.py \
    --d-model 512 \
    --d-ff 2048 \
    --model hybrid \
    --controller transformer \
    --d-ctrl 256 \
    --max-depth 32 \
    --injection-mode broadcast_memory \
    --injection-memory-size 16 \
    --injection-n-heads 4 \
    --epochs 3000 \
    --batch-size 768 \
    --lr 3e-4 \
    --warmup-steps 2000 \
    --n-heads 8 \
    --H-cycles 2 \
    --L-cycles 6 \
    --H-layers 2 \
    --L-layers 2 \
    --hrm-grad-style \
    --halt-max-steps 6 \
    --eval-interval 25 \
    --dropout 0.039 \
    --wandb \
    --project feature-injection-ablation \
    --run-name broadcast-memory-halt6 \
    --resume "eranbt92-open-university-of-israel/feature-injection-ablation/hybrid-transformer-best:latest" \
    --download

In [ ]:
!python experiments/sudoku_poh_benchmark.py \
    --d-model 512 \
    --d-ff 2048 \
    --model hybrid \
    --controller transformer \
    --d-ctrl 256 \
    --max-depth 32 \
    --injection-mode broadcast_memory \
    --injection-memory-size 16 \
    --injection-n-heads 4 \
    --epochs 4500 \
    --batch-size 768 \
    --lr 3e-4 \
    --warmup-steps 2000 \
    --n-heads 8 \
    --H-cycles 4 \
    --L-cycles 12 \
    --H-layers 2 \
    --L-layers 2 \
    --hrm-grad-style \
    --halt-max-steps 8 \
    --eval-interval 25 \
    --dropout 0.039 \
    --wandb \
    --project feature-injection-ablation \
    --run-name broadcast-memory-halt8 \
    --resume "eranbt92-open-university-of-israel/feature-injection-ablation/hybrid-transformer-best:latest" \
    --download